In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import pickle
import sklearn

In [2]:
with open('results/tweet_vectors_supervised.pickle', 'rb') as handle:
    tweet_vectors = pickle.load(handle)

In [3]:
tweet_annotation_df_next = pd.read_csv('tweet_groudtruth_annonation_next.csv',encoding='utf-8-sig')
tweet_annotation_df = pd.read_csv('tweet_groudtruth_annonation.csv',encoding='utf-8-sig')
tweet_annotation = pd.concat([tweet_annotation_df, tweet_annotation_df_next])

In [4]:
X,y = [],[]
annotated_tweets = list(set(tweet_annotation['tweet_id']))
for idx,row in tweet_annotation.iterrows():
    y.append(row['human_annotation'])
    X.append(tweet_vectors[int(row['tweet_id'][:-2])])
X,y = np.array(X),np.array(y)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) #Train test split

In [10]:
from sklearn.model_selection import cross_validate,cross_val_score
clf = RandomForestClassifier(random_state=0)
scoring = ['accuracy','precision','recall','precision_macro','recall_macro','f1_macro','precision_weighted','recall_weighted','f1_weighted','roc_auc']
scores = cross_validate(clf, X, y, scoring=scoring, cv=10, verbose=2, n_jobs=None)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.6s
[CV]  ................................................................
[CV] ................................................. , total=   0.7s
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.5s
[CV]  ................................................................
[CV] ................................................. , total=   0.6s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.7s finished


In [11]:
# print(scores)
for k,v in scores.items():
    print("Average " + k + " = " + str(v.mean()))

Average fit_time = 0.5593947887420654
Average score_time = 0.020790982246398925
Average test_accuracy = 0.788
Average test_precision = 0.8630190694658777
Average test_recall = 0.8560455192034139
Average test_precision_macro = 0.7165816628971723
Average test_recall_macro = 0.7180868621658095
Average test_f1_macro = 0.7137298208553615
Average test_precision_weighted = 0.7902641581796916
Average test_recall_weighted = 0.788
Average test_f1_weighted = 0.7866815688228124
Average test_roc_auc = 0.8624648940438414


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

n_estimators = [50, 60,70, 80, 90, 100]
max_depth = [10,15,20]
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [1, 2, 5, 10]
hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)
rf_features = RandomForestClassifier()
clf_rf_features = GridSearchCV(rf_features, hyperF, scoring = ['accuracy','precision','recall','f1_macro','roc_auc'],cv = 10, verbose = 3, n_jobs = -1)
clf_rf_features.fit(X, y)

best_params = clf_rf_features.best_params_
val_score = clf_rf_features.best_score_
print(best_params)
print("VALIDATION SCORE =", val_score)

clf_predictions = clf_rf_features.predict(X_test)
print(classification_report(y_test,clf_predictions))

best_rf = RandomForestClassifier(max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=80)
best_rf.fit(X_train, y_train)
best_rf_predictions = best_rf.predict(X_test)

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Compute fpr, tpr, thresholds and roc auc
fpr, tpr, thresholds = roc_curve(y_test, best_rf_predictions)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

In [8]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])